In [ ]:
#Import necessary libraries
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph , MessagesState, START, END
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from dotenv import load_dotenv
import os

In [3]:
#Step 1: Load environment variables
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
#Step 2: Define the tool to get weather information
@tool
def get_weather(location: str):
    """Fetch the current weather for a specific location."""
    weather_data = {
        "New York": "Sunny, 25°C",
        "London": "Cloudy, 18°C",
        "Tokyo": "Rainy, 22°C"
    }
    return weather_data.get(location, "Weather data not available for this location.")


#Step 3: Initialize the LLM and bind the tool
tool_node = ToolNode([get_weather], handle_tool_errors=False)
model = ChatOpenAI(model="gpt-4o-mini", temperature=0, openai_api_key=OPENAI_API_KEY).bind_tools([get_weather])


#Step 4: Function to handle user queries and process llm + tool results
def call_llm(state: MessagesState):
    messages = state["messages"]
    #The LLM will decide decide if it should invoke a tool based on the user query
    response = model.invoke(messages[-1].content)
    #If the LLM generate a tool call handle it
    if response.tool_calls:
        tool_result = tool_node.invoke({"messages":[response]})
        #Append the tools output to the response messages
        tool_message = tool_result["messages"][-1].content
        response.content += f"\n\nTool Output: {tool_message}"
    return{"messages": [response]}  

#Step 5: Create the LangGraph workflow
workflow = StateGraph(MessagesState)

#Step 6: Add the LLM node to the workflow
workflow.add_node("call_llm", call_llm)

#Step 7: Define edges to control the flow 
workflow.add_edge(START, "call_llm")
workflow.add_edge("call_llm", END)

#Step 8: Compile the workflow
app = workflow.compile()

#Step 9: Function to interact with the agent continuously
def interact_with_agent():
    while True:
        user_input = input("User: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Exiting the agent. Goodbye!")
            break
        #Prepare the user input for processing
        input_message = {"messages": [("human", user_input)]}
        #Process the input throgh the workflow and run the response
        for chunk in app.stream(input_message,stream_mode="values"):
            chunk ["messages"][-1].pretty_print()  #Display the final response from the agent


#Step 10: Start interacting with the agent
if __name__ == "__main__":
    interact_with_agent()